<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-05/CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch

from google.colab import drive

drive.mount('/content/drive')

im_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256,256)),torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Train/Train', transform = im_transform)
train_loader = torch.utils.data.DataLoader(train_dat, batch_size = 32, shuffle = True)
valid_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Validation/Validation', transform = im_transform)
valid_loader = torch.utils.data.DataLoader(valid_dat, batch_size=len(valid_dat), shuffle = True)
test_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Test/Test', transform = im_transform)
test_loader = torch.utils.data.DataLoader(test_dat)


ValueError: mount failed

In [ ]:
from torch import nn

class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 128, kernel_size=(3,3), stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(128, 128, kernel_size=(3,3), stride = 1, padding = 1)
    self.conv3 = nn.Conv2d(128, 256, kernel_size=(3,3), stride = 1, padding = 1)
    self.pool = nn.MaxPool2d((2,2))
    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(256*32*32, 1024)
    self.fc2 = nn.Linear(1024, 128)
    self.fc3 = nn.Linear(128, 3)
    self.sm = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))
    x = self.pool(self.relu(self.conv3(x)))
    x = torch.flatten(x, 1)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.relu(self.fc3(x))
    return self.sm(x)




In [ ]:
from torch import optim
from sklearn.metrics import accuracy_score
from tqdm import tqdm

epochs = 30
lr = 0.001
model = CNN()
# model.to(torch.device('cuda'))
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr)

for e in range(epochs):
  model.train()
  losses = []
  for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
    optimizer.zero_grad()
    # x = x.to(torch.device('cuda'))
    # y = y.to(torch.device('cuda'))
    pred = model(x)
    loss = loss_fn(pred, y)
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
  print("Epoch: {} | Loss: {:.4f}".format(e,np.mean(losses)))
  best_pred = 0
  model.eval()
  with torch.no_grad():
    for (x, y) in valid_loader:
      # x = x.to(torch.device('cuda'))
      pred = torch.argmax(model(x),1)
      print("Validation Accuracy {:.4f}".format(accuracy_score(torch.Tensor.cpu(pred), torch.Tensor.cpu(y))))
      if accuracy_score(torch.Tensor.cpu(pred), torch.Tensor.cpu(y)) > best_pred:
        best_pred = accuracy_score(torch.Tensor.cpu(pred), torch.Tensor.cpu(y))
        torch.save(model, '/content/drive/MyDrive/Colab Notebooks/best_CNN.pt')